In [ ]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
# from torchtext.data.functional import to_map_style_dataset
# from torch.utils.data import DataLoader
# from torchtext.vocab import build_vocab_from_iterator
# import torchtext.datasets as datasets
# import spacy
# import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [ ]:
# Some convenience helper functions used throughout the notebook
def is_interactive_notebook():
    return __name__ == "__main__"

def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)

def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)

class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None
    def step(self):
        None
    def zero_grad(self, set_to_none=False):
        None

class DummyScheduler:
    def step(self):
        None

#### Encoder-Decoder

In [ ]:
from turtle import forward
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architectire. Base for this and many other models
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequence."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

#### Define standard linear + softmax generation step

In [ ]:
class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)
    
    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

In [ ]:
log_softmax(torch.ones([2,4]), dim=-1)  # dim=-1按最后一维方向来(行方向)

#### Encoder and Decoder Stacks 

##### Encoder 
The encoder is composed of a stack of $N=6$ identical layers

![图片](https://github.com/harvardnlp/annotated-transformer/raw/debc9fd747bb2123160a98046ad1c2d4da44a567//images/ModalNet-21.png)

In [ ]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
[copy.deepcopy([1,2]) for _ in range(3)]

In [ ]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)      # 6层layer堆叠成layers
        self.norm = LayerNorm(layer.size)   # LayerNorm

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

We employ a residual connection around each of the two sub-layers, followed by layer normalization.

$a_2 * \frac{x-mean}{std+eps} + b_2$

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6): # features是单个特征向量长度
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
x = torch.ones(1,64,512)
ln = LayerNorm(512)
ln(x)

In [ ]:
x = torch.ones(2,3,4)
mean = x.mean(dim=-1, keepdim=True)
mean
# keepdim=True时，输出与输入维度相同，仅仅是输出在求范数的维度上元素个数变为1

#### Residual Connection between sublayers
$LayerNorm(x+Sublayer(x))$ 

To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimension $d_{model}=512$.

In [ ]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
        # 这里的sublayer是在以后调用时才传入的，一般是具有forward方法的module或可以对x进行操作的函数
        "Apply residual connection to any sublayer with the same size."
        return self.norm(x + self.dropout(sublayer(x)))

Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-wise fully connected feed-forward network.

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        # 先通过sublayer[0]即attention模块
        # lambda匿名函数，可以对x进行attention操作，输出处理后的向量.
        # attention模块输入参数为x和mask
        x = self.sublayer[0](x, lambda x: self.self_attn(x,x,x,mask))
        # 然后通过PFFN输出encoder的最终结果
        return self.sublayer[1](x, self.feed_forward)

In [ ]:
x = lambda a : a + 10
print(x(5))

##### Decoer
The decoder is also composed of a stack of $N=6$ identical layers.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In addition to the two sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack. Similar to the encoder, we employ residual connections around each of the sub-layers, followed by layer normalization.

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x,x,x,tgt_mask))
        # encoder的输出作为k,v
        # decoder第一个attention的输出作为q
        x = self.sublayer[1](x, lambda x: self.src_attn(x,m,m,src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [216]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.uint8)
    return subsequent_mask == 0
subsequent_mask(4)

tensor([[[ True, False, False, False],
         [ True,  True, False, False],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]]])

#### 可视化掩码

In [ ]:
def example_mask():
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Subsequent Mask": subsequent_mask(20)[0][x, y].flatten(),
                    "Window": y,
                    "Masking": x,
                }
            )
            for y in range(20)
            for x in range(20)
        ]
    )

    return (
        alt.Chart(LS_data)
        .mark_rect()
        .properties(height=250, width=250)
        .encode(
            alt.X("Window:O"),
            alt.Y("Masking:O"),
            alt.Color("Subsequent Mask:Q", scale=alt.Scale(scheme="viridis")),
        )
        .interactive()
    )
show_example(example_mask)

#### Attention 
We call our particular attention "Scaled Dot-Product Attention".The input consists of queries and keys of dimension $d_k$, and values of dimension $d_v$. We compute the dot products of the query with all keys, divide each by $\sqrt{d_k}$, and apply a $softmax$ function to obtain the weights on the values.

![图片](https://github.com/harvardnlp/annotated-transformer/raw/debc9fd747bb2123160a98046ad1c2d4da44a567//images/ModalNet-19.png)

In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix $Q$. The keys and values are also packed together into matrices $K$ and $V$. We compute the matrix of outputs as:
$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$

通过除以$\sqrt{d_k}$规整dot products的结果是为了防止点积结果太大导致softmax结果太大，最终导致某些区域梯度趋于0

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    # 注意力分数
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [ ]:
q = torch.randn(3,512)    # mxn
k = torch.randn(3,512)    # mxn
v = torch.randn(3,256)    # mxk
output, attn_scores = attention(q,k,v)  # attention输出矩阵mxk, 注意力得分矩阵mxm
output.shape, attn_scores.shape

#### Multi-head attention
多头注意力机制允许模型从不同的特征表示子空间中的不同位置注意到不同的信息，语义信息更为丰富。 
多头注意力机制就是先把输入向量投影到$d_{model}/head_num$维，然后通过attentio模块输出$d_{model}/head_num$维的向量，最后再在通道维度上concat还原为$d_{model}$维。
![图片](https://github.com/harvardnlp/annotated-transformer/raw/debc9fd747bb2123160a98046ad1c2d4da44a567//images/ModalNet-20.png)

形式化表达如下：
$Multihead(Q,K,V) = concat(head_1, head_2, ..., head_h)W^O, where \ head_i = Attention(QW^Q_i, KW^K_i, VW^V_i)$ 

其中投影矩阵的尺寸分别为$W^Q_i\in{R^{d_{model}\times{d_k}}}$, $W^K_i\in{R^{d_{model}\times{d_k}}}$, $W^V_i\in{R^{d_{model}\times{d_v}}}$, $W^O_i\in{R^{hd_v\times{d_{model}}}}$ 

我们使用$h=8$(8个头)，对于每个head，我们使用$d_k=d_v=d_{model}/h=64$

多头注意力的计算参数量和单头差不多

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0  # 在表达式条件为false时触发异常
        # 保证d_k永远等于d_v
        self.d_k = d_model // h  # d_k = 512 / 8 = 64
        self.h = h               # h = 8
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            # -1表示其他维数都明确知道数值，但该维度不确定，让程序自己决定
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]
        # query, key, value = [
        #     lin(x).view(nbatches, 8, -1, 64)
        #     for lin, x in zip(self.linears, (query, key, value))
        # ]
        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )
        # 3) "Concat" using a view and apply a final linear.
        x = (x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k))
        del query  # 删除变量，减小内存消耗
        del key
        del value
        return self.linears[-1](x)

##### 测试一下Multi-head attention

In [ ]:
linears = clones(nn.Linear(512, 512), 4)
query = torch.randn(1,3,512)
key = torch.randn(1,3,512)
value = torch.randn(1,3,512)
nbatches = query.size(0)
print("query经过线性投影后的大小",linears[0](query).view(1, 8, -1, 64).shape)
query, key, value = [
            lin(x).view(nbatches, 8, -1, 64)
            for lin, x in zip(linears, (query, key, value))
        ]
print("线性投影后的qkv大小:",query.shape, key.shape, value.shape)
x, attn_scores = attention(query, key, value, mask=None, dropout=nn.Dropout())
print("多头注意力输出向量大小:",x.shape, attn_scores.shape)
x = x.view(nbatches, -1, 512)
print("concat后的向量大小:",x.shape)
print("linear层后的向量大小:",linears[-1](x).shape)

#### Position-wise Feed-Forward Networks

In [ ]:
class PositionwiseFeedForward(nn.Module):
    "Implements PFFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)  # 512x2048
        self.w_2 = nn.Linear(d_ff, d_model)  # 2048x512
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

#### Embeddings and Softmax

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [ ]:
x = nn.Embedding(5000, 512)
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])  # 2个句子
x(segments).shape  # 2个句子，每个句子用8个1x512的向量表示

#### Positional Encoding
$PE(pos,2i)=sin(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$
$PE(pos,2i+1)=cos(\frac{pos}{10000^{\frac{2i}{d_{model}}}})$ 

其中i=0,1,2,...,255

In [ ]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)  # 训练时不会更新pe参数

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [ ]:
pe = torch.zeros(5000, 512)
position = torch.arange(0, 5000).unsqueeze(1) # 在第1维升维
div_term = torch.exp(torch.arange(0, 512, 2) * -(math.log(10000.0) / 512)) # 1x256
pe[:, 0::2] = torch.sin(position * div_term) # 5000行，每行的偶数维替换为sin
pe[:, 1::2] = torch.cos(position * div_term) # 5000行，每行的奇数维替换为cos
pe = pe.unsqueeze(0)  # 在第0维升维
pe[:, :].shape

In [ ]:
def example_positional():
    pe = PositionalEncoding(20, 0)
    y = pe.forward(torch.zeros(1, 100, 20))
    data = pd.concat(
        [
            pd.DataFrame(
                {
                    "embedding": y[0, :, dim],
                    "dimension": dim,
                    "position": list(range(100)),
                }
            )
            for dim in [4, 5, 6, 7]
        ]
    )
    return (
        alt.Chart(data)
        .mark_line()
        .properties(width=800)
        .encode(x="position", y="embedding", color="dimension:N")
        .interactive()
    )
show_example(example_positional)

### Full Model

In [ ]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )
    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [ ]:
class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)
    
    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

In [241]:
test_model = make_model(11, 11, 2) # src_vocab=11, tgt_vocab=11, N=6
test_model.eval()
src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
src_mask = torch.ones(1, 1, 10)

memory = test_model.encode(src, src_mask)  # torch.Size([1, 10, 512])
ys = torch.zeros(1, 1).type_as(src)
for i in range(9):
    out = test_model.decode(
        memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
    )
    print(subsequent_mask(ys.size(1)).type_as(src.data))
    print(out[:, -1].shape)
    prob = test_model.generator(out[:, -1])  # 取最后一行的输出作为预测向量(1x512)
    print(prob.shape)
    _, next_word = torch.max(prob, dim=1)
    next_word = next_word.data[0]
    print(next_word)
    ys = torch.cat(
        [ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1
    )
    print(ys,"\n-----------------------")
# print("Example Untrained Model Prediction:", ys)

# def run_tests():
#     for _ in range(10):
#         inference_test()
# show_example(run_tests)
class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)
    
    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

tensor([[[1]]])
torch.Size([1, 512])
torch.Size([1, 11])
tensor(1)
tensor([[0, 1]]) 
-----------------------
tensor([[[1, 0],
         [1, 1]]])
torch.Size([1, 512])
torch.Size([1, 11])
tensor(2)
tensor([[0, 1, 2]]) 
-----------------------
tensor([[[1, 0, 0],
         [1, 1, 0],
         [1, 1, 1]]])
torch.Size([1, 512])
torch.Size([1, 11])
tensor(7)
tensor([[0, 1, 2, 7]]) 
-----------------------
tensor([[[1, 0, 0, 0],
         [1, 1, 0, 0],
         [1, 1, 1, 0],
         [1, 1, 1, 1]]])
torch.Size([1, 512])
torch.Size([1, 11])
tensor(5)
tensor([[0, 1, 2, 7, 5]]) 
-----------------------
tensor([[[1, 0, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [1, 1, 1, 0, 0],
         [1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1]]])
torch.Size([1, 512])
torch.Size([1, 11])
tensor(5)
tensor([[0, 1, 2, 7, 5, 5]]) 
-----------------------
tensor([[[1, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 0],
         [1, 1, 1, 1

In [3]:
import numpy as np

x = np.array([[[1,2,3],[4,5,6]],[[2,1,3],[5,6,6]]])
x[:,-1]

array([[4, 5, 6],
       [5, 6, 6]])